In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 28.7 MB/s 


In [ ]:
import keras_tuner as kt

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
#len(df_numpy[:2500])
scaler1 = scaler.fit(df_numpy[:3000])
df_scalled = scaler1.transform(df_numpy)
#df_scaled_all = scaler.
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
lookback = 30
step = 1
delay = 30
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2500 - delay,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2500-delay,
max_index=3000 - delay,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3000-delay,
max_index=3542-delay,
step=step,
batch_size=batch_size)

In [ ]:
val_steps = (3000-delay) - (2500-delay) - lookback
test_steps = (3542 - delay) - (300 - delay) - lookback

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(240,
  activation='sigmoid',
  dropout=0.15,
  recurrent_dropout=0.15,
  input_shape=(None, float_train_data.shape[-1])))
model.add(layers.Dense(200,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=200,
  epochs=50,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/50
200/200 [==============================] - 53s 253ms/step - loss: 0.1123 - val_loss: 0.1412
Epoch 2/50
200/200 [==============================] - 53s 267ms/step - loss: 0.0863 - val_loss: 0.1177
Epoch 3/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0866 - val_loss: 0.0899
Epoch 4/50
200/200 [==============================] - 51s 253ms/step - loss: 0.0877 - val_loss: 0.1409
Epoch 5/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0863 - val_loss: 0.1175
Epoch 6/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0866 - val_loss: 0.0898
Epoch 7/50
200/200 [==============================] - 50s 251ms/step - loss: 0.0877 - val_loss: 0.1408
Epoch 8/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0863 - val_loss: 0.1179
Epoch 9/50
200/200 [==============================] - 50s 251ms/step - loss: 0.0866 - val_loss: 0.0897
Epoch 10/50
200/200 [==============================] - 50s 252ms/step - l

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

512/512 [==============================] - 25s 49ms/step - loss: 0.0550


In [ ]:
del model

In [ ]:
def model_builder(hp):
  #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e5])
  hp_units = hp.Choice('units LSTM', values=[10,40,80,120])
  #hp_units2 = hp.Int('units Dense', min_value=150, max_value=180,step=5)
  hp_units2 = hp.Choice('units Dense', values = [10,40,80])
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(units=hp_units,
    #activation=hp.Choice('lstm_activation2',values=['relu', 'sigmoid','softmax']),
    activation='sigmoid',
    recurrent_dropout=0.1,
    #dropout=0.5,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout rate',min_value=0,max_value=0.5,step=0.1)))
  model.add(layers.Dense(units=hp_units2,
    #activation=hp.Choice('dense_activation',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.add(layers.Dense(1,
    ##activation = hp.Choice('last_activ',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='mae')
  return model

In [ ]:
tuner = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=20,
    directory = 'try4')

Best 240 - 200 0.4 dropout 0.1 rec dropout

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 5 Complete [00h 05m 25s]
val_loss: 0.035131532698869705

Best val_loss So Far: 0.03444389998912811
Total elapsed time: 00h 23m 15s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
120               |120               |units LSTM
80                |80                |units Dense
0.4               |0.2               |Dropout rate

Epoch 1/10
200/200 [==============================] - 34s 159ms/step - loss: 0.1429 - val_loss: 0.2883
Epoch 2/10
200/200 [==============================] - 32s 158ms/step - loss: 0.0886 - val_loss: 0.1259
Epoch 3/10
200/200 [==============================] - ETA: 0s - loss: 0.0810

KeyboardInterrupt: ignored

new test required, since this is not using the data

In [ ]:
len(float_game_data)

542

In [ ]:
correct_picks = 0
value = 0
for i in range(542-30): #len(float_train_data - lookback - delay
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[30][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 1000 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 1000 * (last_val - actual_val)
  else:
    value -= 1000 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

In [ ]:
correct_picks

247

In [ ]:
value

-3776.523530483246

version 2 updates the model

In [ ]:
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(180,
    activation='sigmoid',
    dropout=0.15,
    recurrent_dropout=0.15,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(100,
    activation='sigmoid',))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

In [ ]:
len(float_train_data)

931

In [ ]:
correct_picks = 0
value = 0
for i in range(0,542-30):
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[29][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 1000 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 1000 * (last_val - actual_val)
  else:
    value -= 1000 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

  if i % 30 == 0:
    del model
    del train_gen
    model = return_model()
    train_gen = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - delay - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)
    model.fit(train_gen,
      steps_per_epoch=200,
      epochs=50,
      validation_data=val_gen,
      validation_steps=val_steps)

Epoch 1/50
200/200 [==============================] - 56s 254ms/step - loss: 0.1143 - val_loss: 0.2022
Epoch 2/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0844 - val_loss: 0.1014
Epoch 3/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0701 - val_loss: 0.0715
Epoch 4/50
200/200 [==============================] - 50s 251ms/step - loss: 0.0624 - val_loss: 0.0972
Epoch 5/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0584 - val_loss: 0.0880
Epoch 6/50
200/200 [==============================] - 52s 260ms/step - loss: 0.0556 - val_loss: 0.0782
Epoch 7/50
200/200 [==============================] - 50s 251ms/step - loss: 0.0548 - val_loss: 0.0791
Epoch 8/50
200/200 [==============================] - 50s 251ms/step - loss: 0.0534 - val_loss: 0.0845
Epoch 9/50
200/200 [==============================] - 50s 252ms/step - loss: 0.0524 - val_loss: 0.0759
Epoch 10/50
200/200 [==============================] - 51s 253ms/step - l

In [ ]:
correct_picks

NameError: ignored

In [ ]:
value

NameError: ignored